<a href="https://colab.research.google.com/github/avkaz/DeepLearningPetIdentification/blob/preprocess_pipeline/preprop_pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
import json
from PIL import Image, ExifTags, ImageDraw
import requests
import io
import tensorflow as tf
import numpy as np
import tensorflow_hub as hub
import matplotlib.pyplot as plt

# Metadata Fetching
def get_data():
    """
    Fetches and parses JSON data from the given URL.

    Returns:
        dict: The parsed JSON data as a Python dictionary.
    """
    url = "https://raw.githubusercontent.com/avkaz/DeepLearningPetIdentification/main/pets_db.json"

    try:
        response = requests.get(url)
        response.raise_for_status()
        data = response.json()
        return data
    except requests.RequestException as e:
        print(f"An error occurred while fetching data: {e}")
        raise
    except json.JSONDecodeError as e:
        print(f"An error occurred while parsing JSON: {e}")
        raise

# Model Loading
detector = None
MODEL_URL = "https://tfhub.dev/tensorflow/ssd_mobilenet_v2/fpnlite_320x320/1"





Total number of "pets", where metadata: "images" is empty.

In [20]:
# Count and print entries with empty 'images' list
empty_images_count = sum(1 for entry in metadata.values() if not entry.get("images"))

# Print the result
print(f"Number of entries with empty images: {empty_images_count}")



Number of entries with empty images: 1722


In [13]:
def load_detector_model():
    global detector
    if detector is None:
        print("Uploading model...")
        detector = hub.load(MODEL_URL).signatures['serving_default']
        print("Model loaded successfully.")

In [ ]:
# Metadata Verification
def verify_metadata(metadata):
    """
    Verifies metadata integrity by checking for missing or inconsistent entries.

    Args:
        metadata (dict): The metadata dictionary to verify.
    """
    for key, value in metadata.items():
        if not isinstance(value, dict) or "label" not in value or "images" not in value:
            print(f"Warning: Incomplete metadata for key {key}: {value}")

# Orientation Correction
def fix_orientation(image):
    try:
        for orientation in ExifTags.TAGS.keys():
            if ExifTags.TAGS[orientation] == 'Orientation':
                break
        exif = image._getexif()
        if exif is not None:
            orientation = exif.get(orientation)
            if orientation == 3:
                image = image.rotate(180, expand=True)
            elif orientation == 6:
                image = image.rotate(270, expand=True)
            elif orientation == 8:
                image = image.rotate(90, expand=True)
    except (AttributeError, KeyError, IndexError):
        pass
    return image

# Cropping and Resizing
def crop_and_resize(image, bounding_box, target_size):
    image = tf.convert_to_tensor(image, dtype=tf.float32)
    x1, y1, x2, y2 = bounding_box
    image = tf.strided_slice(image, [int(y1), int(x1), 0], [int(y2), int(x2), 3])
    image = tf.image.resize(image, target_size)
    return image

# Pet Detection
def detect_pet(image):
    load_detector_model()
    input_tensor = tf.image.resize(image, [640, 640]) / 255.0
    input_tensor = tf.expand_dims(input_tensor, axis=0)
    input_tensor_uint8 = tf.cast(input_tensor * 255.0, tf.uint8)

    result = detector(tf.convert_to_tensor(input_tensor_uint8))
    result = {key: value.numpy() for key, value in result.items()}

    if 'detection_classes' in result and 'detection_scores' in result:
        detected_classes = result['detection_classes']
        detected_boxes = result['detection_boxes']
        detected_scores = result['detection_scores']
        pet_classes = [b"Cat", b"Dog", b"Animal"]

        for idx in range(len(detected_classes[0])):
            detected_class = detected_classes[0][idx]
            detected_score = detected_scores[0][idx]
            detected_box = detected_boxes[0][idx]

            if detected_class in pet_classes and detected_score > 0.5:
                return detected_box
    return None

# Detection Visualization
def visualize_detection(image, bounding_box, title="Detection Visualization"):
    """
    Displays an image with a bounding box for detected pets.

    Args:
        image: The input image (PIL Image).
        bounding_box: Bounding box coordinates (x1, y1, x2, y2).
        title: Title for the visualization.
    """
    if bounding_box:
        draw = ImageDraw.Draw(image)
        x1, y1, x2, y2 = [int(coord) for coord in bounding_box]
        draw.rectangle([x1, y1, x2, y2], outline="red", width=3)
    plt.imshow(image)
    plt.title(title)
    plt.axis("off")
    plt.show()

# Download and Preprocess Image
def download_and_preprocess_image(url, target_size=(224, 224), visualize=False):
    response = requests.get(url)
    image_bytes = response.content
    pil_image = Image.open(io.BytesIO(image_bytes))
    pil_image = fix_orientation(pil_image)

    image = tf.convert_to_tensor(np.array(pil_image), dtype=tf.float32) / 255.0
    bounding_box = detect_pet(image)

    if bounding_box is not None:
        image = crop_and_resize(image, bounding_box, target_size)
        if visualize:
            visualize_detection(pil_image, bounding_box)
    else:
        image = tf.image.resize_with_crop_or_pad(image, target_size[0], target_size[1])
    return image

This function processes image data from a metadata dictionary and prepares it for training in a machine learning model using TensorFlow.

In [ ]:
# Dataset Creation
#def preprocess_pipeline(metadata, target_size=(224, 224), visualize=False):
 #   images = []
  #  labels = []
   # for key, entry in metadata.items():
    #    image_url = entry["images"]
     #   label = entry["Plemeno"]
      #  image = download_and_preprocess_image(image_url, target_size, visualize)
       # images.append(image)
        #labels.append(label)
   # return tf.data.Dataset.from_tensor_slices((images, labels))

def preprocess_pipeline(metadata, target_size=(224, 224), visualize=False):
    images = []
    labels = []
    for key, entry in metadata.items():
        image_urls = entry["images"]

        # Create a label (list of characteristics)
        label = [
            entry.get("Plemeno", "Unknown"),
            entry.get("Věk", "Unknown"),
            entry.get("Barva", "Unknown"),
            entry.get("Velikost", "Unknown")
        ]

        # Process all the images in the 'images' list
        for image_url in image_urls:
            image = download_and_preprocess_image(image_url, target_size, visualize)
            images.append(image)
            labels.append(label)  # Append the same label for each image

    return tf.data.Dataset.from_tensor_slices((images, labels))

This function will display a specified number of sample images from the dataset, with the corresponding labels shown as the image title.

In [ ]:
# Dataset Sample Visualization
def visualize_dataset_sample(dataset, num_samples=5):
    for image, label in dataset.take(num_samples):
        plt.imshow(image.numpy())
        plt.title(f"Label: {label.numpy().decode()}")
        plt.axis("off")
        plt.show()

Split data into parts: training, validation and test data.

---



In [ ]:
def split_data(dataset, val_split=0.1, test_split=0.1):
    total_size = len(dataset)
    val_size = int(val_split * total_size)
    test_size = int(test_split * total_size)

    val_dataset = dataset.take(val_size)
    test_dataset = dataset.skip(val_size).take(test_size)
    train_dataset = dataset.skip(val_size + test_size)

    return train_dataset, val_dataset, test_dataset


This function prepares the dataset by batching it into groups of a specified size. It can also shuffle the dataset before batching and prefetch it to optimize performance.

In [ ]:
# Dataset Batching
def batch_dataset(dataset, batch_size=32, shuffle=True):
    if shuffle:
        dataset = dataset.shuffle(buffer_size=1000)
    dataset = dataset.batch(batch_size).prefetch(tf.data.AUTOTUNE)
    return dataset

In [ ]:
if __name__ == "__main__":
    print("Fetching metadata...")
    metadata = get_data()
    verify_metadata(metadata)

    print("Creating dataset...")
    dataset = preprocess_pipeline(metadata, visualize=True)

    print("Visualizing dataset samples...")
    visualize_dataset_sample(dataset, num_samples=3)


TODO: label encoding, image augmentation?, performance optimization - parallel processing ...